# Assignment 2 - RAG

Parts which require your interaction are marked with `TODO:`

In [2]:
# Install the relevant dependencies
!pip3 install datasets sentence_transformers tqdm numpy

Imagine your task is to build a question-answering (QA) system for a company. You are given a language model and have to create this product out of it.
The requirements of the system need to adapt very quickly to the new data without training.
For this, we will use **Retrieval Augmented Generation (RAG)**.
The company insists you use their in-house LM model trained on multiple tasks, a _flan-t5-small_.
You can test its QA functionality by asking the question _"When ETH was founded?"_:

In [3]:
# Example inference with the model.
# TODO: run me to test the environment

from transformers import pipeline
vanilla_qa_pipe = pipeline("text2text-generation", model="google/flan-t5-small", device="cuda:0", truncation=True)

QUESTION = "QUESTION: When was ETH founded?"

vanilla_qa_pipe(f"{QUESTION} ANSWER:", max_new_tokens=10)[0]["generated_text"]

Device set to use cuda:0


'1897'

In [4]:
vanilla_qa_pipe(f"""
        CONTEXT: ETH Zurich (German: Eidgenoessische Technische Hochschule Zurich; English:
        Federal Institute of Technology Zurich) is a public research university in Zurich,
        Switzerland. Founded in 1854 with the stated mission to educate engineers and scientists,
        the university focuses primarily on science, technology, engineering, and mathematics. It
        consistently ranks among the top universities in the world and its 16 departments span a
        variety of disciplines and subjects.
        {QUESTION}
        ANSWER:",
    """,
    max_new_tokens=10
)[0]["generated_text"]

'1854'

The first output is 1897, which is incorrect.

This is not a problem, we can use RAG to automatically provide the passage from an [external source](https://en.wikipedia.org/wiki/ETH_Zurich) and make the model answer. Concatenating the first paragraph from Wikipedia to the question makes the model yield the correct answer 1854.

In [5]:
# Define model function; do not modify
from typing import List

def rag_qa_pipe(question: str, passages: List[str]) -> str:
    """
    Define the RAG pipeline which concatenates passages to the question.
    :param question: Question text.
    :param passages: Relevant text passages.
    :return: Generated text from the pipeline.
    """
    passages = "\n".join([f"CONTEXT: {c}" for c in passages])
    return vanilla_qa_pipe(f"{passages}\nQUESTION: {question}\nANSWER: ", max_new_tokens=10)[0]["generated_text"]

To make sure you understand the function `rag_qa_pipe`, ask some question without and with some relevant context.

In [6]:
# TODO: use rag_qa_pipeline some random question that you might have just to test this function

print(rag_qa_pipe("When is the first book of Harry potter released?", []))
print(rag_qa_pipe("When is the first book of Harry potter released?", ["Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. Since the release of the first novel, Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity and commercial success worldwide."]))

September 27, 2017
26 June 1997


Start with the provided model and the first 500 questions from the validation part of the _SQuAD_ dataset. The dataset has a ground truth Wikipedia passage linked to it and you can directly use it.

Then, compute the QA performance of the model with and without prepended passage using `rag_qa_pipe(question, passages)`.

Report the average case-sensitive answer exact match (model output is identical to the gold answer, EM) and case-insensitive [answer F1 scores](https://kierszbaumsamuel.medium.com/f1-score-in-nlp-span-based-qa-task-5b115a5e7d41) (F1) for both setups.
Because each question has multiple possible answers, take the maximum score for a model answer across all gold answers.

In [ ]:
# baseline model evaluation
# TODO: the this cell requires <30 new lines

import tqdm
import numpy as np
from collections import Counter
from datasets import load_dataset
dataset = load_dataset("rajpurkar/squad")

def metric_exact_match(ans_pred: str, ans_true: str) -> float:
    """
    Case-sensitive answer exact match, model output is identical to the gold answer.
    :param ans_pred: Predicted answer
    :param ans_true: Ground truth answer
    :return: 1. if the answers are the same, 0. otherwise
    """
    # TODO: ~1 line
    return float(ans_pred == ans_true)

def metric_f1(ans_pred: str, ans_true: str) -> float:
    """
    Case-insensitive answer F1 score.
    :param ans_pred: Predicted answer.
    :param ans_true: Ground truth answer.
    :return: F1 score between the predicted and ground truth answers.
    """
    # TODO: ~10 lines
    gt_toks = ans_true.split()
    pred_toks = ans_pred.split()
    comm = Counter(gt_toks) & Counter(pred_toks)
    snum = sum(comm.values())
    if snum == 0:
        return 0.0
    precision = snum / len(pred_toks)
    recall = snum / len(gt_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

for line in tqdm.tqdm(dataset["validation"].select(range(500))):
    # hint: use `line["question"]`, `line["context"]`, and `line["answers"]`
    # TODO: run with and without prepended passage
    # Run the model without prepended passage
    pred_without_passage = vanilla_qa_pipe(f"QUESTION: {line['question']} ANSWER:", max_new_tokens=10)[0]["generated_text"]

    # Run the model with prepended passage
    pred_with_passage = vanilla_qa_pipe(f"CONTEXT: {line['context']}\nQUESTION: {line['question']}\nANSWER:", max_new_tokens=10)[0]["generated_text"]

    # Compute exact match and F1 scores
    em_without_passage = max(metric_exact_match(pred_without_passage, ans) for ans in line["answers"]["text"])
    f1_without_passage = max(metric_f1(pred_without_passage, ans) for ans in line["answers"]["text"])

    em_with_passage = max(metric_exact_match(pred_with_passage, ans) for ans in line["answers"]["text"])
    f1_with_passage = max(metric_f1(pred_with_passage, ans) for ans in line["answers"]["text"])

    # Store the results
    if "results" not in locals(): 
        results = {"em_without_passage": [], "f1_without_passage": [], "em_with_passage": [], "f1_with_passage": []}
    results["em_without_passage"].append(em_without_passage)
    results["f1_without_passage"].append(f1_without_passage)
    results["em_with_passage"].append(em_with_passage)
    results["f1_with_passage"].append(f1_with_passage)

# TODO: Print mean of the exact match and mean of F1 scores for the model with and without prepended passage
print("Mean exact match without passage:", np.mean(results["em_without_passage"]))
print("Mean f1 score without passage:", np.mean(results["f1_without_passage"]))
print("Mean exact match with passage:", np.mean(results["em_with_passage"]))
print("Mean f1 score with passage:", np.mean(results["f1_with_passage"]))

100%|██████████| 500/500 [01:21<00:00,  6.10it/s]

Mean exact match without passage: 0.020942408376963352
Mean f1 score without passage: 0.07713648577522923
Mean exact match with passage: 0.7085514834205934
Mean f1 score with passage: 0.7728355607936759


You will likely see improvements in scores by providing a passage to the model.

In contrast to the previous evaluation, during inference in a real world scenario, we do not have access to the ground truth passage.
All we have access to is the question from a user.
Luckily, the company is providing you with an unstructured knowledge base. This could be the whole of Wikipedia but in our scenario, we use all the passages from the SQuAD dataset and shuffle them to remove any existing structure.

In [ ]:
import random

kb = list(set(dataset["validation"]["context"]))

# make sure that there is no remaining structure
random.Random(42).shuffle(kb)
print(len(kb), "passages in the knowledge base")

2067 passages in the knowledge base


Now whenever we receive a question, we need to find the relevant passage(s) from the knowledge base and put it in the model input.
This is a non-trivial task and a whole research field of Information Retrieval is devoted to it.


We are going to convert all the knowledge base passages into vectors using TF-IDF and the provided embedding model ([bert-base-nli-max-tokens](https://huggingface.co/sentence-transformers/bert-base-nli-max-tokens)).
The model inference is already implemented for you but you need to fill in all the functions in the `KnowledgeBase` class.
You will need to implement the retrieval, the distance metrics, and the three similarity metrics (Euclidean, cosine, inner product).

We need to build an abstraction for the knowledge base. It needs to support:
- adding new keys (vectors) and their corresponding values
- retrieving the closest key given one, based on 3 vector distance metrics

The implementation does not need to be efficient.

Hint: it's ok to just add all the elements to a list and on retrieval sort the list by the distance.

In [11]:
# Knowledge base building. This cell requires <20 new lines.
from typing import Literal, List, Any

Vec = List
Val = Any

class KnowledgeBase:
    def __init__(self, dim: int):
        """
        Initialize a knowledge base with a given dimensionality.
        :param dim: the dimensionality of the vectors to be stored
        """
        # TODO: initialize a persistent structure, such as a simple list
        self.data = []
        self.dim = dim

    def add_item(self, key: Vec, val: Val):
        """
        Store the key-value pair in the knowledge base.
        :param key: key
        :param val: value
        """
        # TODO: add to the persistent structure
        self.data.append((key, val))
        
    def retrieve(
        self, key: Vec, metric: Literal['l2', 'cos', 'ip'], k: int = 1
    ) -> List[Val]:
        """
        Retrieve the top k values from the knowledge base given a key and similarity metric.
        :param key: key
        :param metric: Similarity metric to use.
        :param k: Top k similar items to retrieve.
        :return: List of top k similar values.
        """
        # TODO: retrieve the k closest vectors and return their corresponding values
        # Hint: this does not have to be efficient, feel free to just sort the whole persistent structure and return the top k
        if metric == 'l2':
            similarity_func = self._sim_euclidean
        elif metric == 'cos':
            similarity_func = self._sim_cosine
        elif metric == 'ip':
            similarity_func = self._sim_inner_product
        else:
            raise ValueError(f"Unknown metric: {metric}")

        # Compute similarity for all items and sort by similarity
        similarities = []
        for stored_key, stored_val in self.data:
            similarity = similarity_func(key, stored_key)
            similarities.append((similarity, stored_val))
        sorted_items = sorted(similarities, key=lambda x: x[0]) if metric == 'l2' else sorted(similarities, key=lambda x: x[0], reverse=True)
        # Return the top k values
        return [val for _, val in sorted_items[:k]]

    @staticmethod
    def _sim_euclidean(a: Vec, b: Vec) -> float:
        """
        Compute Euclidean (L2) distance between two vectors.
        :param a: Vector a
        :param b: Vector b
        :return: Similarity score
        """
        # hint: use numpy
        # TODO: compute the Euclidean distance between two vectors
        a, b = np.array(a), np.array(b)
        distance = np.sqrt(np.sum((a - b) ** 2))
        return distance

    @staticmethod
    def _sim_cosine(a: Vec, b: Vec) -> float:
        """
        Compute the cosine similarity between two vectors.
        :param a: Vector a
        :param b: Vector b
        :return: Similarity score
        """
        # hint: use numpy
        # TODO: compute the cosine distance between two vectors
        a, b = np.array(a), np.array(b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        if norm_a == 0 or norm_b == 0:
            return 0.0
        similarity = np.dot(a, b) / (norm_a * norm_b)
        return similarity

    @staticmethod
    def _sim_inner_product(a: Vec, b: Vec) -> float:
        """
        Compute the inner product between two vectors.
        :param a: Vector a
        :param b: Vector b
        :return: Similarity score
        """
        # hint: use numpy
        # TODO: compute the iner product distance between two vectors
        a, b = np.array(a), np.array(b)
        similarity = np.dot(a, b)
        return similarity


In [12]:
# Build knowledge base index
# In ideal case this does not need to be changed and can just be run.
# Make modifications if you feel they are necessary.

from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

# # Sparse retrieval using TF-IDF - vectorize with tfidf and retrieve
vectorizer = TfidfVectorizer(max_features=768, norm=None)
kb_vectorized = np.asarray(vectorizer.fit_transform([x for x in kb]).todense())
kb_index_tfidf = KnowledgeBase(dim=768)
for passage_index, passage_embd in enumerate(kb_vectorized):
    kb_index_tfidf.add_item(passage_embd.squeeze(), passage_index)

# Dense retrieval using Sentence Transformers
model_embd = SentenceTransformer("bert-base-nli-mean-tokens").to("cuda:0")
kb_index_embd = KnowledgeBase(dim=768)
for passage_index, passage_embd in enumerate(tqdm.tqdm(kb)):
    kb_index_embd.add_item(model_embd.encode(passage_embd).squeeze(), passage_index)

100%|██████████| 2067/2067 [00:27<00:00, 73.98it/s]


For the same first 500 questions from the validation split evaluate how often is the retrieved passage the correct one (formally Recall@1) or among the top 5 retrieved (Recall@5).
Perform the retrieval with three distance metrics: euclidean distance, cosine distance, and inner product. The result for this should be 12 numbers.

In production, you receive a question from the user and to answer it, you need to first retrieve the relevant passage(s), pass it to the model, and only then generate the answer.

Evaluate the model performance with passages retrieved by TFIDF and EMBD vectorization.
Consider top-1 and top-5 passages.
This time use only case-insensitive F1.
The result for this cell should be |vectorizations $\times$ passage sizes $\times$ distance metrics = 2 x 2 x 3 = 12 numbers.

Answer the following questions:
* Provide one potential advantage and two potential disadvantages of using multiple retrieved passages?
* Describe one approach to detect if none of the retrieved passages is relevant to the user question.

TODO:
### Answer to question1
* Advantage: improving the coverage of answers: using multiple passages increases the likelihood that at least one passage contains the correct answer, this is especially useful when the top-1 passage might be incorrect due to noise or ambiguity. Disadvantages: increasing computational cost; increasing the risk of introducing irrelevant or misleading information.
### Answer to question2
* Assign a relevance score to each retrieved passage based on its similarity to the question (e.g., cosine similarity for embeddings). Set a predefined threshold for relevance (e.g., based on validation data where passages below a certain similarity score are unlikely to contain the answer). If none of the retrieved passages exceed this threshold, flag them as irrelevant.



In [13]:
# In ideal case this does not need to be changed and can just be run.
# Make modifications if you feel they are necessary.

retrieval_results = {"recall_at_1": {}, "recall_at_5": {}}
for metric in ["l2", "cos", "ip"]:
    print(metric.upper())
    retrieval_results["recall_at_1"][metric] = []
    retrieval_results["recall_at_5"][metric] = []
    
    for line in tqdm.tqdm(dataset["validation"].select(range(500))):
        # TODO: evaluate the retrieval
        # TODO: store RAG model output
        # This requires <30 new lines
        # Retrieve top-1 and top-5 passages using the current metric
        top_1_passages = kb_index_embd.retrieve(model_embd.encode(line["question"]).squeeze(), metric=metric, k=1)
        top_5_passages = kb_index_embd.retrieve(model_embd.encode(line["question"]).squeeze(), metric=metric, k=5)

        # Check if the correct passage is in the retrieved passages
        correct_passage = line["context"]
        recall_at_1 = int(correct_passage in [kb[idx] for idx in top_1_passages])
        recall_at_5 = int(correct_passage in [kb[idx] for idx in top_5_passages])

        # Store the results
        retrieval_results["recall_at_1"][metric].append(recall_at_1)
        retrieval_results["recall_at_5"][metric].append(recall_at_5)

    # Print the average recall scores for the current metric
    print(f"Recall@1: {np.mean(retrieval_results['recall_at_1'][metric])}")
    print(f"Recall@5: {np.mean(retrieval_results['recall_at_5'][metric])}")

L2


100%|██████████| 500/500 [00:28<00:00, 17.53it/s]


Recall@1: 0.234
Recall@5: 0.522
COS


100%|██████████| 500/500 [00:35<00:00, 14.04it/s]


Recall@1: 0.238
Recall@5: 0.556
IP


100%|██████████| 500/500 [00:18<00:00, 27.56it/s]

Recall@1: 0.21
Recall@5: 0.554


Answer the following questions about similarity metrics:
* Compare and contrast the three metrics, what they might be influenced by, and their advantages and disadvantages.
* Consider the scenario if the vectors in the knowledge base were normalized so that $|x|_2 = 1$. What would the results look like? Hint: look at the formulas with this vector assumption.

TODO:
### Answer to question1
* Euclidean distance: measures straight-line distance, sensitive to magnitude and dimensionality. Influenced by vector magnitude, noise, high-dimensional effects. Advantage: effetive when magnitude matters. Disadvantages: poor in high dimensions, sensitive to magnitude, more computational cost.
* Cosine similarity: measures angular distance, ignores magnitude. Influenced by vector direction, normalization precision and sparsity. Advantages: robust for embeddings, effective for semantic similarity. Disadvantages: more computational cost, less effective for sparse vectors with low overlap.
* Inner product: measures dot product, sensitive to both magnitude and direction. Influenced by vector scale, magnitude and sparsity. Advantages: computationally efficient, effective with normalization. Disadvantages: Performance depends on how the embedding model scales vectors; inconsistent scaling can reduce effectiveness.
### Answer to question2 
* When the vectors in the knowledge base were normalized so that $|x|_2 = 1$, all metrics will be equivalent in ranking, inner product metric will be the most efficient one. The result like retrieval performance (e.g.,Recall@1 and Recall@5) will be identical across metrics.

Lastly, it is a good practice to analyze failure cases of your solution to better understand the pipeline.
Find the first example of each and compute how often the situation happens (percentage). Use the maximum exact match to determine correctness and L2 + embedding for retrieval.

- For top-1: The retrieved passage is **correct** but the model is **not correct**.
- For top-1: The retrieved passage is **not correct** but the model is **still correct**.
- For top-5: One of the retrieved passages is the **correct** one but the model is **not correct**.
- For top-1: Without retrieved passage is the model **correct** but with the passage the model becomes **incorrect**.
- For top-1: Without retrieved passage is the model **incorrect** and with the passage the model becomes **incorrect** but in a different way (different answer).

In [14]:
# compute the 5 phenomena statistics (relative frequency) and find examples

# TODO: <30 lines
for line in tqdm.tqdm(dataset["validation"].select(range(500))):
  # Retrieve top-1 passage using L2 embedding
  top_1_passage = kb_index_embd.retrieve(model_embd.encode(line["question"]).squeeze(), metric="l2", k=1)[0]
  retrieved_passage = kb[top_1_passage]

  # Check the phenomena
  correct_passage = line["context"]
  pred_with_passage = rag_qa_pipe(line["question"], [retrieved_passage])
  pred_without_passage = rag_qa_pipe(line["question"], [])

  em_with_passage = max(metric_exact_match(pred_with_passage, ans) for ans in line["answers"]["text"])
  em_without_passage = max(metric_exact_match(pred_without_passage, ans) for ans in line["answers"]["text"])

  # Phenomenon 1: Retrieved passage is correct but model is not correct
  if retrieved_passage == correct_passage and em_with_passage == 0:
    if "phenomenon_1" not in locals():
      phenomenon_1 = []
    phenomenon_1.append(line)

  # Phenomenon 2: Retrieved passage is not correct but model is still correct
  if retrieved_passage != correct_passage and em_with_passage == 1:
    if "phenomenon_2" not in locals():
      phenomenon_2 = []
    phenomenon_2.append(line)

  # Phenomenon 3: One of the top-5 retrieved passages is correct but model is not correct
  top_5_passages = kb_index_embd.retrieve(model_embd.encode(line["question"]).squeeze(), metric="l2", k=5)
  top_5_retrieved = [kb[idx] for idx in top_5_passages]
  if correct_passage in top_5_retrieved and em_with_passage == 0:
    if "phenomenon_3" not in locals():
      phenomenon_3 = []
    phenomenon_3.append(line)

  # Phenomenon 4: Without retrieved passage model is correct but with passage it becomes incorrect
  if em_without_passage == 1 and em_with_passage == 0:
    if "phenomenon_4" not in locals():
      phenomenon_4 = []
    phenomenon_4.append(line)

  # Phenomenon 5: Without retrieved passage model is incorrect and with passage it becomes incorrect in a different way
  if em_without_passage == 0 and em_with_passage == 0 and pred_with_passage != pred_without_passage:
    if "phenomenon_5" not in locals():
      phenomenon_5 = []
    phenomenon_5.append(line)

# Compute relative frequencies
total_examples = len(dataset["validation"].select(range(500)))
print("Phenomenon 1 frequency:", len(phenomenon_1) / total_examples * 100, "%")
print("Phenomenon 2 frequency:", len(phenomenon_2) / total_examples * 100, "%")
print("Phenomenon 3 frequency:", len(phenomenon_3) / total_examples * 100, "%")
print("Phenomenon 4 frequency:", len(phenomenon_4) / total_examples * 100, "%")
print("Phenomenon 5 frequency:", len(phenomenon_5) / total_examples * 100, "%")

100%|██████████| 500/500 [02:42<00:00,  3.08it/s]

Phenomenon 1 frequency: 5.6000000000000005 %
Phenomenon 2 frequency: 4.0 %
Phenomenon 3 frequency: 32.800000000000004 %
Phenomenon 4 frequency: 0.6 %
Phenomenon 5 frequency: 68.0 %


A client is complaining that the model answers incorrectly the question _"Who is the current Governor of Victoria?"_.
1. Show your model output to this question with top-1 retrieved passage using any metric.
2. Show which top-1 context is retrieved by L2 embd.

Hint for the correct answer, see: [en.wikipedia.org/wiki/Premier_of_Victoria](https://en.wikipedia.org/wiki/Premier_of_Victoria).

In [15]:
QUESTION = "Who is the premier of Victoria?"
# TODO: < 20 lines
# Retrieve the top-1 passage using the specified metric cosine similarity
top_1_passage = kb_index_embd.retrieve(model_embd.encode(QUESTION).squeeze(), metric="cos", k=1)[0]
retrieved_passage = kb[top_1_passage]
# Generate the model's answer using the retrieved passage
answer = rag_qa_pipe(QUESTION, [retrieved_passage])
# Print the retrieved passage and the model's answer
print("Retrieved passage by using metric cosine similarity:", retrieved_passage)
print("Model's answer:", answer)

# Retrieve the top-1 context using 2 embedding
top_1_context_l2 = kb_index_embd.retrieve(model_embd.encode(QUESTION).squeeze(), metric="l2", k=1)[0]
retrieved_context_l2 = kb[top_1_context_l2]

# Print the retrieved context
print("Top-1 context retrieved by l2 embedding:", retrieved_context_l2)

Retrieved passage by using metric cosine similarity: The Premier of Victoria is the leader of the political party or coalition with the most seats in the Legislative Assembly. The Premier is the public face of government and, with cabinet, sets the legislative and political agenda. Cabinet consists of representatives elected to either house of parliament. It is responsible for managing areas of government that are not exclusively the Commonwealth's, by the Australian Constitution, such as education, health and law enforcement. The current Premier of Victoria is Daniel Andrews.
Model's answer: Daniel Andrews
Top-1 context retrieved by l2 embedding: The Premier of Victoria is the leader of the political party or coalition with the most seats in the Legislative Assembly. The Premier is the public face of government and, with cabinet, sets the legislative and political agenda. Cabinet consists of representatives elected to either house of parliament. It is responsible for managing areas of

Answer the following questions:
* Provide a reason why your model is giving the incorrect answer. (information tracing)
* Propose a way by which this could be remedied. (information editing)


TODO:
### Answer to question1
* The model is giving the incorrect answer because the retrieved passage contains outdated information. The knowledge base used by the RAG system has not been updated with the latest information about the Premier of Victoria. The model is accurately answering the question based on the information it has, but that information is no longer correct.

### Answer to question2
* In order to remedy this issue, the first intuitive way will be updating the knowledge base, regularly update the knowledge base with the latest information from reliable sources. This could involve scraping Wikipedia pages, using news APIs, or other methods to ensure the data is current.

Note on compute: the GPU time of the gold solution is ~15 minutes. If your solution requires much more compute (e.g. hours), then you are likely doing something incorrectly.